In [1]:
import time
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import csv
import logging
from dotenv import load_dotenv

In [2]:
def create_logfile():
    date_time = datetime.datetime.today().strftime('%d-%b-%y_%H:%M:%S')
    logfile = f"log/{date_time}.log"
    logging.basicConfig(filename=logfile, filemode='w', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)
    logging.info(f'Log file {logfile} created')
    return logging

def create_file(file, logging):
    # delete existing file if re-running
    logging.info("Checking if current daily csv exists...")
    if os.path.exists(file):
        os.remove(file)
        logging.info(f"{file} deleted")
    else:
        logging.info(f"{file} ain't exist")
    
    # create file and add header
    logging.info("Creating daily csv file...")
    header = ['date_time', 'search_keyword', 'search_count', 'job_id', 'job_title', 'company', 'location', 'remote', 'update_time', 'applicants', 'job_pay', 'job_time', 'job_position', 'company_size', 'company_industry', 'job_details']
    with open(file, 'w') as f:
        w = csv.writer(f)
        w.writerow(header)
        logging.info(f"{file} created")

def login(logging):
    url_login = "https://www.linkedin.com/"

    # pulls login information from file called '.env' 
    # this file added to .gitignore so login details not shared
    load_dotenv()
    # .env file is of structure:
    # LINKEDIN_USERNAME=email@gmail.com
    # LINKEDIN_PASSWORD=password

    LINKEDIN_USERNAME = os.getenv('LINKEDIN_USERNAME')
    LINKEDIN_PASSWORD = os.getenv('LINKEDIN_PASSWORD')

    # setup chrome to run headless
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--window-size=1920,1080")

    # login to LinkedIn
    logging.info(f"Logging in to LinkedIn as {LINKEDIN_USERNAME}...")
    wd = webdriver.Chrome(executable_path='./chromedriver', options=chrome_options)
    wd.get(url_login)
    wd.find_element_by_id("session_key").send_keys(LINKEDIN_USERNAME)
    wd.find_element_by_id("session_password").send_keys(LINKEDIN_PASSWORD)
    wd.find_element_by_xpath("//button[@class='sign-in-form__submit-button']").click()

    # random confirm acount information pop up that may come up
    try: 
        wd.find_element_by_xpath("//button[@class='primary-action-new']").click()
    except:
        pass
    logging.info("Log in complete. Scraping data...")

    return wd

def page_search(wd, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging):
    # wait time for events in seconds
    page_wait = 30
    click_wait = 5
    async_wait = 5

    # when retrying, number of attempts
    attempts = 3

    # navigate to search page
    url_search = f"https://www.linkedin.com/jobs/search/?f_TPR={search_posted}&f_WRA={search_remote}&geoId=103644278&keywords={search_keyword}&location={search_location}&start={search_page}"
    wd.get(url_search)
    time.sleep(page_wait) # add sleep so don't get caught

    # find the number of results 
    search_count = wd.find_element_by_xpath("//small[@class='display-flex t-12 t-black--light t-normal']").text
    search_count = int(search_count.split(' ')[0].replace(',', ''))  # get number before space & remove comma (ex. "1,245 results")
    logging.info(f"Loading page {round(search_page/25) + 1} of {round(search_count/25)} for {search_keyword}'s {search_count} results...")

    # get all the job_id's for xpath for current page to click each element
    # running into errors with slow load (11-Aug)
    for attempt in range(attempts):
        try:
            search_results = wd.find_element_by_xpath("//ul[@class='jobs-search-results__list list-style-none']").find_elements_by_tag_name("li")
            result_ids = [result.get_attribute('id') for result in search_results if result.get_attribute('id') != '']
            break
        except:
            time.sleep(click_wait) # wait a few attempts, if not throw an exception and then skip to next page

    # cycle through each job_ids and steal the job data...muhahaha!
    list_jobs = [] #initate a blank list to append each page to
    for id in result_ids:
        try:
            job = wd.find_element_by_id(id) 
            job_id = job.get_attribute("data-occludable-entity-urn").split(":")[-1]
            # select a job and start extracting information
            wd.find_element_by_xpath(f"//div[@data-job-id={job_id}]").click()
        except:
            continue
            # exception likely to job deleteing need to go to next id

        for attempt in range(attempts):
            try:
                # from analysis 3 attempts at 5 second waits gets job titles 99.99% of time (11-Aug)
                job_title = wd.find_element_by_xpath("//h2[@class='t-24 t-bold']") # keep having issues with finding element
                job_title = job_title.text
                break
            except:
                job_title = ''
                time.sleep(click_wait)
        
        # Having issues finding xpath of company (Added 11-Aug)
        for attempt in range(attempts):
            try:
                job_top_card1 = wd.find_element_by_xpath("//span[@class='jobs-unified-top-card__subtitle-primary-grouping mr2 t-black']").find_elements_by_tag_name("span")
                company = job_top_card1[0].text
                location = job_top_card1[1].text
                if len(job_top_card1) > 2: # only displays remote if selected, otherwise only 2 elements in list
                    remote = job_top_card1[2].text
                else:
                    remote = ''
                break
            except:
                company = ''
                location = ''
                remote = ''
                time.sleep(click_wait)
        
        for attempt in range(attempts):
            try:
                #multiple issues with job_top_card2 loading
                job_top_card2 = wd.find_element_by_xpath("//span[@class='jobs-unified-top-card__subtitle-secondary-grouping t-black--light']").find_elements_by_tag_name("span")
                update_time = job_top_card2[0].text
                applicants = job_top_card2[1].text.split(' ')[0]
                break
            except: 
                update_time = '' # after #attempts leave as blank and move on
                applicants = '' # after #attempts leave as blank and move on
                time.sleep(click_wait)

        # Due to (slow) ASYNCHRONOUS updates, need wait times to get job_info
        job_time = '' # assigning as blanks as not important info and can skip if not obtained below
        job_position = ''
        job_pay = ''
        for attempt in range(attempts):
            try:
                # 1 - make sure HTML element is loaded
                element = WebDriverWait(wd, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@class='mt5 mb2']/div[1]")))
                # 2 - make sure text is loaded
                try:
                    job_info = element.text
                    if job_info != '':
                        # seperate job information on time requirements and position
                        job_info = job_info.split(" · ")
                        if len(job_info) == 1: # only one item means its job _time
                            job_pay = ''
                            job_time = job_info[0]
                            job_position = ''
                        elif (len(job_info) >= 2) and ("$" in job_info[0]): # if has money symbol then seperate
                            job_pay = job_info[0]
                            job_time = job_info[1]
                            if(len(job_info)>= 3): # check if job_info is required
                                job_position = job_info[2]
                            else:
                                job_position = ''
                        else: # else condition satisifies the last condition
                            job_time = job_info[0]
                            job_position = job_info[1]
                            job_pay = ''
                        break
                    else:
                        time.sleep(async_wait)
                except:
                    # error means page didn't load so try again
                    time.sleep(async_wait)
            except:
                # error means page didn't load so try again
                time.sleep(async_wait)

        # get company details and seperate on size and industry
        company_size = '' # assigning as blanks as not important info and can skip if not obtained below
        company_industry = ''
        job_details = ''      
        for attempt in range(attempts):
            try:
                company_details = wd.find_element_by_xpath("//div[@class='mt5 mb2']/div[2]").text
                if " · " in company_details:
                    company_size = company_details.split(" · ")[0]
                    company_industry = company_details.split(" · ")[1]
                else:
                    company_size = company_details
                    company_industry = ''
                job_details = wd.find_element_by_id("job-details").text.replace("\n", " ")
                break
            except: 
                time.sleep(click_wait)

        # append (a) line to file
        date_time = datetime.datetime.now().strftime("%d%b%Y-%H:%M:%S")
        search_keyword = search_keyword.replace("%20", " ")
        list_job = [date_time, search_keyword, search_count, job_id, job_title, company, location, remote, update_time, applicants, job_pay, job_time, job_position, company_size, company_industry, job_details]
        list_jobs.append(list_job)

    with open(file, "a") as f:
        w = csv.writer(f)
        w.writerows(list_jobs)
        list_jobs = []
    
    logging.info(f"Page {round(search_page/25) + 1} of {round(search_count/25)} loaded for {search_keyword}")
    search_page += 25

    return search_page, search_count, url_search


In [3]:
# create logging file
logging = create_logfile()

# create daily csv file
date = datetime.date.today().strftime('%d-%b-%y')
file = f"output/{date}.csv"
create_file(file, logging)

# login to linkedin and assign webdriver to variable
wd = login(logging)

# URL search terms focusing on what type of skills are required for Data Analyst & Data Scientist
search_keywords = ['Data Analyst', 'Data Scientist', 'Data Engineer']
    # Titles to remove as search is too long
    # ['Business Analyst', 'Operations Analyst', 'Marketing Analyst', 'Product Analyst',
    # 'Analytics Consultant', 'Business Intelligence Analyst', 'Quantitative Analyst',  'Data Architect',
    # 'Data Engineer', 'Machine Learning Engineer', 'Machine Learning Scientist']
search_location = "United%20States"
search_remote = "true" # filter for remote positions
search_posted = "r86400" # filter for past 24 hours

# Counting Exceptions
exception_first = 0
exception_second = 0

for search_keyword in search_keywords:
    search_keyword = search_keyword.lower().replace(" ", "%20")

# Loop through each page and write results to csv
    search_page = 0 # start on page 1
    search_count = 1 # initiate search count until looks on page
    while (search_page < search_count) and (search_page != 1000):
        # Search each page and return location after each completion
        try:
            search_page, search_count, url_search = page_search(wd, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging)
        except Exception as e:
            logging.error(f'(1) FIRST exception for {search_keyword} on {search_page} of {search_count}, retrying...')
            logging.error(f'Current URL: {url_search}')
            logging.error(e)
            logging.exception('Traceback ->')
            exception_first += 1
            time.sleep(5) 
            try:
                search_page, search_count, url_search = page_search(wd, search_location, search_keyword, search_remote, search_posted, search_page, search_count, file, logging)
                logging.warning(f'Solved Exception for {search_keyword} on {search_page} of {search_count}')
            except Exception as e:
                logging.error(f'(2) SECOND exception remains for {search_keyword}. Skipping to next page...')
                logging.error(f'Current URL: {url_search}')
                logging.error(e)
                logging.exception('Traceback ->')
                search_page += 25 # skip to next page to avoid entry
                exception_second += 1
                logging.error(f'Skipping to next page for {search_keyword}, on {search_page} of {search_count}...')

# close browser
wd.quit()

logging.info(f'LinkedIn data scraping complete with {exception_first} first and {exception_second} second exceptions')
logging.info(f'Regard all further alarms...')